<a href="https://colab.research.google.com/github/HYUNMIN-HWANG/Colab/blob/main/DACON_vision2/vision2_0226_2_contour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('./MyDrive')


Mounted at ./MyDrive


#dirty_mnist_2nd 이미지들을 글자 하나씩 잘라보자

In [17]:
import cv2 as cv
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os

In [32]:
test_img=glob('/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/*.png')
print(len(test_img))   # 5000
test_img = sorted(test_img)
print(test_img)

5000
['/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50000.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50001.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50002.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50003.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50004.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50005.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50006.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50007.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50008.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50009.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50010.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50011.png', '/content/MyDrive/MyDrive/Colab/DACON_vision2/

In [33]:
n = 50000

for img in test_img :    
    print(img)
    # 이미지 불러오기
    img_color = cv.imread(img)
    
    # 블러처리
    img_gray = cv.cvtColor(img_color, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(img_gray, (3,3), cv.BORDER_DEFAULT)
    # plt.imshow(blur)

    # 이진화
    ret, img_binary = cv.threshold(blur, 170, 255, cv.THRESH_BINARY)
    # plt.imshow(blur)

    # 부식
    kernel = np.ones((3,3), np.uint8)
    erode = cv.erode(img_binary, kernel, iterations = 1)  

    # 노이즈 제거
    morph = cv.morphologyEx(erode, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE, (2,2)), iterations=2)
    # plt.imshow(morph)
    
    # 외곽선 검출
    contours , hierarchy = cv.findContours(morph , cv.RETR_EXTERNAL , cv.CHAIN_APPROX_NONE)

    color = cv.cvtColor(morph, cv.COLOR_GRAY2BGR)           # 이진화 이미지를 color이미지로 복사(확인용)
    cv.drawContours(color , contours , -1 , (0,255,0),3)    # 초록색으로 외곽선을 그려준다.

    bR_arr = []
    digit_arr = []
    digit_arr2 = []
    count = 0

    for i in range(len(contours)) :
        bin_tmp = morph.copy()
        x,y,w,h = cv.boundingRect(contours[i])
        bR_arr.append([x,y,w,h])

    #x값을 기준으로 배열을 정렬
    bR_arr = sorted(bR_arr, key=lambda num : num[0], reverse = False)
    # print(bR_arr[:5])
    # print(len(bR_arr))  # 

    #작은 노이즈데이터 버림,사각형그리기
    for x,y,w,h in bR_arr :
        tmp_y = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[0]
        tmp_x = bin_tmp[y-2:y+h+2,x-2:x+w+2].shape[1]
        if  tmp_x and tmp_y > 15 :
            count += 1 
            cv.rectangle(color,(x-2,y-2),(x+w+2,y+h+2),(0,0,255),1)
            digit_arr.append(bin_tmp[y-2:y+h+2,x-2:x+w+2])
            # print(bin_tmp[y-2:y+h+2,x-2:x+w+2])
    digit_arr2.append(digit_arr)
    # print(len(digit_arr))        # 13
    # print(len(digit_arr2))       # 1
    # plt.imshow(color)
    # k = cv.waitKey(0)
    # cv.destroyAllWindows()

    # print(digit_arr[0][0])
    # print(digit_arr2[0][3])
    # a = np.array(digit_arr2[0][3])
    # print(a.shape)

    #리스트에 저장된 이미지를 32x32의 크기로 리사이즈해서 순서대로 저장
    for i in range(0,len(digit_arr2)) :
        # print("i ", i)  # 전체 이미지 하나
        
        for j in range(len(digit_arr2[i])) :
            count += 1 
            # print("j ", j)
            if i == 0 :         #1일 경우 비율 유지를 위해 마스크를 만들어 그 위에 얹어줌
                width = digit_arr2[i][j].shape[1]
                # print(width)    # 52
                height = digit_arr2[i][j].shape[0]
                # print(height)   # 54
                if width < height :
                    mask = np.zeros((height,height))
                    tmp = (height - width)/2
                    mask[0:height, int(tmp):int(tmp)+width] = digit_arr2[i][j]
                    digit_arr2[i][j] = cv.resize(mask,(64,64))
                else :
                    mask = np.zeros((width, width))
                    tmp = (width - height)/2
                    mask[int(tmp):int(tmp)+height, 0:width] = digit_arr2[i][j]
                    digit_arr2[i][j] = cv.resize(mask,(64,64))
            else:
                digit_arr2[i][j] = cv.resize(digit_arr2[i][j],(64,64))
            if i == 9 : i = -1
            cv.imwrite('/content/MyDrive/MyDrive/Colab/DACON_vision2/contour/'+str(n)+'_'+str(j)+'.png',digit_arr2[i][j])
    
    n +=1
    if n == 4 :
        break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50000.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50001.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50002.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50003.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50004.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50005.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50006.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50007.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50008.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50009.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50010.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_dirty_mnist_2nd/50011.png
/content/MyDrive/MyDrive/Colab/DACON_vision2/test_di